Firstly we have to intall PostgreSql database adapter(Driver) for the Python programming language(pip install psycopg2-binary)

In [1]:
pip install psycopg2-binary

Note: you may need to restart the kernel to use updated packages.


In [7]:
#1

# Connection Python with Database

import psycopg2 
import getpass  
conn=psycopg2.connect(user='postgres',
                      database='postgres',
                      host='localhost',
                      port='5432',
                      password=getpass.getpass('Insert password: ')
                        )
print(conn.get_dsn_parameters())

Insert password: ········
{'user': 'postgres', 'dbname': 'postgres', 'host': 'localhost', 'port': '5432', 'tty': '', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'}


In [8]:
#2.

# Request needed information from DB

import pandas as pd

query="SELECT * FROM bond_description;"
data=pd.read_sql_query(query,conn)
data

#2934 rows affected

,"ISIN, RegCode, NRDCode",FinToolType,SecurityType,SecurityKind,CouponType,RateTypeNameRus_NRD,CouponTypeName_NRD,HaveOffer,AmortisedMty,MaturityGroup,...,BorrowerName,BorrowerOKPO,BorrowerSector,BorrowerUID,IssuerName,IssuerName_NRD,IssuerOKPO,NumGuarantors,EndMtyDate,Issuer_ID
0,RU000A0JPSD9,Облигация,Корп,Биржевые,Постоянный,None,None,False,False,Краткосрочные,...,АвтоВАЗ,232934.0,Автомобили и запчасти,6270,АвтоВАЗ,None,232934.0,0,2009-05-27,121054
1,RU000A0JNYT8,Облигация,Корп,Классические,Переменный,None,None,True,False,Краткосрочные,...,Алькор и Ко,17295933.0,Ритейл,89216,Алькор и Ко,None,17295933.0,1,2009-12-08,135812
2,RU000A0JP591,Облигация,Корп,Классические,Переменный,None,None,True,False,Краткосрочные,...,Амазонит,40437333.0,"Химпром, минудобрения",89008,Амазонит,None,40437333.0,1,2010-04-12,135908
3,RU000A0GNTQ0,Облигация,Корп,Классические,Постоянный,None,None,True,True,Краткосрочные,...,Аркада АПК,70202426.0,Пищевая промышленность и с/х,86999,Аркада АПК,None,70202426.0,2,2009-02-20,124453
4,RU000A0JP8H9,Облигация,Корп,Классические,Переменный,None,None,True,False,Краткосрочные,...,Бахетле-1,48694029.0,Ритейл,89584,Бахетле-1,None,48694029.0,4,2010-05-19,135963
5,RU000A0JNZX7,Облигация,Корп,Классические,Переменный,None,None,True,False,Краткосрочные,...,Бразерс и Компания,45155875.0,"Путешествия, досуг",89089,Бразерс и Компания,None,45155875.0,1,2009-12-25,135852
6,RU000A0JNYC4,Облигация,Корп,Классические,Переменный,None,None,True,False,Среднесрочные,...,Банк Интеза,17512023.0,Банки,84890,Банк Интеза,None,17512023.0,1,2011-11-30,96666
7,RU000A0GKF03,Облигация,Корп,Классические,Переменный,None,None,True,False,Краткосрочные,...,Белгранкорм,45810417.0,Пищевая промышленность и с/х,88313,Белгранкорм,None,45810417.0,1,2008-12-02,85608
8,RU000A0JPM48,Облигация,Корп,Классические,Переменный,None,None,True,False,Среднесрочные,...,ВТБ 24,20606880.0,Банки,6537,ВТБ 24,None,20606880.0,0,2013-02-05,115160
9,RU000A0BBB37,Облигация,Муни,Классические,Постоянный,None,None,False,False,Среднесрочные,...,Минфин Московской Обл,2278662.0,None,93813,Московская Обл Прав,None,22036.0,0,2009-04-21,851037
